In [293]:
%load_ext autoreload
%autoreload

from deep_nmmo.envs.team_based_env.teams.custom_team import CustomTeam
from deep_nmmo.utils import get_class_from_path, get_module_from_path
from deep_nmmo.envs.team_based_env.env_configs.custom_competition_config import CustomCompetitionConfig
from deep_nmmo.envs.team_based_env.loops.utils import init_env_params, reset_teams, reset_env

import nmmo
from nmmo import config
from nmmo.io import action
from nmmo import scripting, material, Serialized
from nmmo.systems import skill, item
from nmmo.lib import colors
from nmmo import action as Action


import neurips2022nmmo
from neurips2022nmmo.scripted import baselines
from neurips2022nmmo import Team
from neurips2022nmmo import CompetitionConfig, scripted, RollOut, TeamBasedEnv
from neurips2022nmmo.scripted import attack, move

import ray
ray.shutdown()
ray.init()

from ray.rllib.agents import ppo
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog

# from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.policy.policy import PolicySpec

from ray.rllib.utils.annotations import override
from ray.rllib.utils.typing import (
    # AgentID,
    # EnvCreator,
    # EnvID,
    # EnvType,
    MultiAgentDict,
    # MultiEnvDict,
)

import gym

import numpy as np

import copy
from collections import defaultdict

from IPython.display import display

from typing import Dict, Any, Type, List

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2022-09-04 20:01:03,181	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


### TODO

- Create RLLibScriptedBaseline class to wrap around neurips2022nmmo.scripted.baselines agents so that agents are compatible with RLlib environments. Should inherit from RLlib Policy base class
     - May need to have wrapper class for Team as well. Need to ensure all this still works when making submission

In [337]:
%autoreload

from ray.rllib.policy.policy import Policy
        
# class RLlibNMMOPolicy(Policy):
#     """Pass NMMO agent to make compatible with RLlib envs."""
#     def __init__(self, 
#                  observation_space, 
#                  action_space, 
#                  config,
#                  idx,
#                  nmmo_agent_cls, # can be string path to cls or raw cls
#                  **kwargs):
#         self.observation_space = observation_space
#         self.action_space = action_space
#         self.config = config
#         if isinstance(nmmo_agent_cls, str):
#             self.nmmo_agent = get_class_from_path(nmmo_agent_cls)(config=config, idx=idx)
#         else:
#             self.nmmo_agent = nmmo_agent_cls(config=config, idx=idx)
#         # self.nmmo_agent = nmmo_agent
#         # self.action_space.seed(_seed)
        
        

#     def compute_actions(self,
#                         obs_batch,
#                         state_batches=None,
#                         prev_action_batch=None,
#                         prev_reward_batch=None,
#                         info_batch=None,
#                         episodes=None,
#                         **kwargs):
#         """Compute actions on a batch of observations."""
#         # return [self.action_space.sample() for _ in obs_batch], [], {}
#         return [self.nmmo_agent(obs) for obs in obs_batch], [], {}

#     # def compute_single_action(self, obs, *args, **kwargs):
#     #     return self.nmmo_agent(obs)
    
#     def learn_on_batch(self, samples):
#         """No learning."""
#         #return {}
#         pass

#     def get_weights(self):
#         pass

#     def set_weights(self, weights):
#         pass



# class RLlibNMMOPolicy(Policy, nmmo.Agent):
class RLlibNMMOPolicy(Policy):
    """Pass NMMO agent to make compatible with RLlib envs."""
    def __init__(self, 
                 observation_space, 
                 action_space, 
                 config, # must include 'env_config' and 'idx'
                 **kwargs):
        Policy.__init__(self, observation_space=observation_space, action_space=action_space, config={})
        for key in ['env_config', 'idx']:
            if key not in config:
                raise Exception(f'Config must include key {key}')
        self.observation_space = observation_space
        self.action_space = action_space
        self.idx = config['idx']
        self.env_config = config['env_config']
        # if isinstance(nmmo_agent_cls, str):
        #     self.nmmo_agent = get_class_from_path(nmmo_agent_cls)(config=config, idx=idx)
        # else:
        #     self.nmmo_agent = nmmo_agent_cls(config=config, idx=idx)
        
        self.health_max = self.env_config.PLAYER_BASE_HEALTH

        if self.env_config.RESOURCE_SYSTEM_ENABLED:
            self.food_max = self.env_config.RESOURCE_BASE
            self.water_max = self.env_config.RESOURCE_BASE

        self.spawnR = None
        self.spawnC = None

    def compute_single_action(self, *args, **kwargs):
        print(f'COMPUTE SINGLE ACTION CALLED')

    def compute_actions(self,
                        obs_batch,
                        state_batches=None,
                        prev_action_batch=None,
                        prev_reward_batch=None,
                        info_batch=None,
                        episodes=None,
                        **kwargs):
        """Compute actions on a batch of observations."""
        # return [self.action_space.sample() for _ in obs_batch], [], {}
        # return [self.nmmo_agent(obs) for obs in obs_batch], [], {}
        print(f'COMPUTE ACTIONS CALLED')
        
        # HACK: Assume not batching (since not supported by nmmo agents as far as I can see)
        if isinstance(obs_batch, list):
            raise Exception(f'Have not implemented observation batching, currently assumes obs_batch is just obs')
        obs = obs_batch
        
        self.actions = {}

        obs = obs_batch[0]
        print(f'Computing obs from obs:\n{obs}')
        print(f'obs len: {len(obs)}')
        print(f'obs[0] len: {len(obs[0])}')
        print(f'obs shape: {np.array(obs).shape}')
        self.ob = scripting.Observation(self.env_config, obs)
        print(f'Computed obs!!!')
        agent = self.ob.agent

        # Time Alive
        self.timeAlive = scripting.Observation.attribute(
            agent, Serialized.Entity.TimeAlive)

        # Pos
        self.r = scripting.Observation.attribute(agent, Serialized.Entity.R)
        self.c = scripting.Observation.attribute(agent, Serialized.Entity.C)

        #Resources
        self.health = scripting.Observation.attribute(agent,
                                                      Serialized.Entity.Health)
        self.food = scripting.Observation.attribute(agent,
                                                    Serialized.Entity.Food)
        self.water = scripting.Observation.attribute(agent,
                                                     Serialized.Entity.Water)

        #Skills
        self.melee = scripting.Observation.attribute(agent,
                                                     Serialized.Entity.Melee)
        self.range = scripting.Observation.attribute(agent,
                                                     Serialized.Entity.Range)
        self.mage = scripting.Observation.attribute(agent,
                                                    Serialized.Entity.Mage)
        self.fishing = scripting.Observation.attribute(
            agent, Serialized.Entity.Fishing)
        self.herbalism = scripting.Observation.attribute(
            agent, Serialized.Entity.Herbalism)
        self.prospecting = scripting.Observation.attribute(
            agent, Serialized.Entity.Prospecting)
        self.carving = scripting.Observation.attribute(
            agent, Serialized.Entity.Carving)
        self.alchemy = scripting.Observation.attribute(
            agent, Serialized.Entity.Alchemy)

        #Combat level
        # TODO: Get this from agent properties
        self.level = max(self.melee, self.range, self.mage, self.fishing,
                         self.herbalism, self.prospecting, self.carving,
                         self.alchemy)

        self.skills = {
            skill.Melee: self.melee,
            skill.Range: self.range,
            skill.Mage: self.mage,
            skill.Fishing: self.fishing,
            skill.Herbalism: self.herbalism,
            skill.Prospecting: self.prospecting,
            skill.Carving: self.carving,
            skill.Alchemy: self.alchemy
        }

        if self.spawnR is None:
            self.spawnR = scripting.Observation.attribute(
                agent, Serialized.Entity.R)
        if self.spawnC is None:
            self.spawnC = scripting.Observation.attribute(
                agent, Serialized.Entity.C)

        # When to run from death fog in BR configs
        self.fog_criterion = None
        if self.env_config.PLAYER_DEATH_FOG is not None:
            step_per_tile = max(1, int(1 / self.env_config.PLAYER_DEATH_FOG_SPEED))
            start_running = self.timeAlive > self.env_config.PLAYER_DEATH_FOG - step_per_tile
            run_now = (self.timeAlive % step_per_tile == 0)
            self.fog_criterion = start_running and run_now
    
    
    

    # def compute_single_action(self, obs, *args, **kwargs):
    #     return self.nmmo_agent(obs)
    
    def learn_on_batch(self, samples):
        """No learning."""
        #return {}
        pass

    def get_weights(self):
        pass

    def set_weights(self, weights):
        pass

    @property
    def policy(self):
        return self.__class__.__name__

    @property
    def forage_criterion(self) -> bool:
        '''Return true if low on food or water'''
        min_level = 7 * self.env_config.RESOURCE_DEPLETION_RATE
        return self.food <= min_level or self.water <= min_level

    def forage(self):
        '''Min/max food and water using Dijkstra's algorithm'''
        move.forageDijkstra(self.env_config, self.ob, self.actions, self.food_max,
                            self.water_max)

    def gather(self, resource):
        '''BFS search for a particular resource'''
        return move.gatherBFS(self.env_config, self.ob, self.actions, resource)

    def explore(self):
        '''Route away from spawn'''
        sz = self.env_config.MAP_SIZE
        centR, centC = sz // 2, sz // 2
        if self.timeAlive < sz // 4:
            move.explore(self.env_config, self.ob, self.actions, self.spawnR,
                         self.spawnC)
        elif self.fog_criterion:
            move.explore(self.env_config, self.ob, self.actions, self.r, self.c)
        else:
            move.explore(self.env_config, self.ob, self.actions, centR, centC)
        #move.explore(self.env_config, self.ob, self.actions, self.spawnR, self.spawnC)

    @property
    def downtime(self):
        '''Return true if agent is not occupied with a high-priority action'''
        return not self.forage_criterion and self.attacker is None

    def evade(self):
        '''Target and path away from an attacker'''
        move.evade(self.env_config, self.ob, self.actions, self.attacker)
        self.target = self.attacker
        self.targetID = self.attackerID
        self.targetDist = self.attackerDist

    def attack(self):
        '''Attack the current target'''
        if self.target is not None:
            assert self.targetID is not None
            style = random.choice(self.style)
            attack.target(self.env_config, self.actions, style, self.targetID)

    def target_weak(self):
        '''Target the nearest agent if it is weak'''
        if self.closest is None:
            return False

        selfLevel = scripting.Observation.attribute(self.ob.agent,
                                                    Serialized.Entity.Level)
        targLevel = scripting.Observation.attribute(self.closest,
                                                    Serialized.Entity.Level)
        population = scripting.Observation.attribute(
            self.closest, Serialized.Entity.Population)

        if population == -1 or targLevel <= selfLevel <= 5 or selfLevel >= targLevel + 3:
            self.target = self.closest
            self.targetID = self.closestID
            self.targetDist = self.closestDist

    def scan_agents(self):
        '''Scan the nearby area for agents'''
        self.closest, self.closestDist = attack.closestTarget(
            self.env_config, self.ob)
        self.attacker, self.attackerDist = attack.attacker(
            self.env_config, self.ob)

        self.closestID = None
        if self.closest is not None:
            self.closestID = scripting.Observation.attribute(
                self.closest, Serialized.Entity.ID)

        self.attackerID = None
        if self.attacker is not None:
            self.attackerID = scripting.Observation.attribute(
                self.attacker, Serialized.Entity.ID)

        self.target = None
        self.targetID = None
        self.targetDist = None

    def adaptive_control_and_targeting(self, explore=True):
        '''Balanced foraging, evasion, and exploration'''
        self.scan_agents()

        if self.attacker is not None:
            self.evade()
            return

        if self.fog_criterion:
            self.explore()
        elif self.forage_criterion or not explore:
            self.forage()
        else:
            self.explore()

        self.target_weak()

    def process_inventory(self):
        if not self.env_config.ITEM_SYSTEM_ENABLED:
            return

        self.inventory = set()
        self.best_items = {}
        self.item_counts = defaultdict(int)

        self.item_levels = {
            item.Hat: self.level,
            item.Top: self.level,
            item.Bottom: self.level,
            item.Sword: self.melee,
            item.Bow: self.range,
            item.Wand: self.mage,
            item.Rod: self.fishing,
            item.Gloves: self.herbalism,
            item.Pickaxe: self.prospecting,
            item.Chisel: self.carving,
            item.Arcane: self.alchemy,
            item.Scrap: self.melee,
            item.Shaving: self.range,
            item.Shard: self.mage
        }

        self.gold = scripting.Observation.attribute(self.ob.agent,
                                                    Serialized.Entity.Gold)

        for item_ary in self.ob.items:
            itm = Item(item_ary)
            cls = itm.cls

            assert itm.cls.__name__ == 'Gold' or itm.quantity != 0
            #if itm.quantity == 0:
            #   continue

            self.item_counts[cls] += itm.quantity
            self.inventory.add(itm)

            #Too high level to equip
            if cls in self.item_levels and itm.level > self.item_levels[cls]:
                continue

            #Best by default
            if cls not in self.best_items:
                self.best_items[cls] = itm

            best_itm = self.best_items[cls]

            if itm.level > best_itm.level:
                self.best_items[cls] = itm

            if __debug__:
                err = 'Key {} must be an Item object'.format(cls)
                assert isinstance(self.best_items[cls], Item), err

    def upgrade_heuristic(self, current_level, upgrade_level, price):
        return (upgrade_level - current_level) / max(price, 1)

    def process_market(self):
        if not self.env_config.EXCHANGE_SYSTEM_ENABLED:
            return

        self.market = set()
        self.best_heuristic = {}

        for item_ary in self.ob.market:
            itm = Item(item_ary)
            cls = itm.cls

            self.market.add(itm)

            #Prune Unaffordable
            if itm.price > self.gold:
                continue

            #Too high level to equip
            if cls in self.item_levels and itm.level > self.item_levels[cls]:
                continue

            #Current best item level
            current_level = 0
            if cls in self.best_items:
                current_level = self.best_items[cls].level

            itm.heuristic = self.upgrade_heuristic(current_level, itm.level,
                                                   itm.price)

            #Always count first item
            if cls not in self.best_heuristic:
                self.best_heuristic[cls] = itm
                continue

            #Better heuristic value
            if itm.heuristic > self.best_heuristic[cls].heuristic:
                self.best_heuristic[cls] = itm

    def equip(self, items: set):
        for cls, itm in self.best_items.items():
            if cls not in items:
                continue

            if itm.equipped:
                continue

            self.actions[Action.Use] = {Action.Item: itm.instance}

            return True

    def consume(self):
        if self.health <= self.health_max // 2 and item.Poultice in self.best_items:
            itm = self.best_items[item.Poultice]
        elif (self.food == 0
              or self.water == 0) and item.Ration in self.best_items:
            itm = self.best_items[item.Ration]
        else:
            return

        self.actions[Action.Use] = {Action.Item: itm.instance}

    def sell(self, keep_k: dict, keep_best: set):
        for itm in self.inventory:
            price = itm.level
            cls = itm.cls

            if cls == item.Gold:
                continue

            assert itm.quantity > 0

            if cls in keep_k:
                owned = self.item_counts[cls]
                k = keep_k[cls]
                if owned <= k:
                    continue

            #Exists an equippable of the current class, best needs to be kept, and this is the best item
            if cls in self.best_items and cls in keep_best and itm.instance == self.best_items[
                    cls].instance:
                continue

            self.actions[Action.Sell] = {
                Action.Item: itm.instance,
                Action.Price: Action.Price.edges[int(price)],
            }

            return itm

    def buy(self, buy_k: dict, buy_upgrade: set):
        if len(self.inventory) >= self.env_config.ITEM_INVENTORY_CAPACITY:
            return

        purchase = None
        best = list(self.best_heuristic.items())
        random.shuffle(best)
        for cls, itm in best:
            #Buy top k
            if cls in buy_k:
                owned = self.item_counts[cls]
                k = buy_k[cls]
                if owned < k:
                    purchase = itm

            #Check if item desired
            if cls not in buy_upgrade:
                continue

            #Check is is an upgrade
            if itm.heuristic <= 0:
                continue

            #Buy best heuristic upgrade
            self.actions[Action.Buy] = {Action.Item: itm.instance}

            return itm

    def exchange(self):
        if not self.env_config.EXCHANGE_SYSTEM_ENABLED:
            return

        self.process_market()
        self.sell(keep_k=self.supplies, keep_best=self.wishlist)
        self.buy(buy_k=self.supplies, buy_upgrade=self.wishlist)

    def use(self):
        self.process_inventory()
        if self.env_config.EQUIPMENT_SYSTEM_ENABLED and not self.consume():
            self.equip(items=self.wishlist)

    








        
class RLlibRandom(RLlibNMMOPolicy):
    def __init__(self,
                 *args,
                 **kwargs):
        RLlibNMMOPolicy.__init__(self, *args, **kwargs)
        
    def compute_actions(self, obs, *args, **kwargs):
        print(f'Computing nmmo agent actions...')
        super().compute_actions(obs)
        print(f'nmmo agent actions computed')

        print(f'Computing random agent actions...')
        move.random(self.config, self.ob, self.actions)
        print(f'Random agent actions computed')
        return self.actions, [], {}


class RLlibMultiAgentTeamBasedEnv(MultiAgentEnv):
    def __init__(self, 
                 env_config,
                 # path_to_env_cls,
                 # path_to_env_config_cls,
                 teams_config):
        '''
        Notes on interacting with _env TeamBasedEnv internally:
            - TeamBasedEnv.players: Dict mapping player ID to player object
            - TeamBasedEnv.player_team_map: Dict mapping player ID to team ID
            - TeamBasedEnv.team_player_map: Dict mapping team ID to list of player IDs
        '''
        # inherit from RLlib multi-agent env
        MultiAgentEnv.__init__(self)
        
        # # init env config
        # env_config = get_class_from_path(path_to_env_config_cls)()
        
        # HACK: init dummy teams and env so can init RLlib policies with obs and action space
        from neurips2022nmmo.scripted import RandomTeam
        dummy_teams = [RandomTeam(team_id=i, env_config=env_config) for i in range(len(list(teams_config.keys())))]
        for i, team in enumerate(dummy_teams):
            class Agent(nmmo.Agent):
                name = f'{team.id}'
                policy = f'{team.id}'
            env_config.PLAYERS[i] = Agent
        dummy_env = get_class_from_path(path_to_env_cls)(env_config)
        _ = dummy_env.reset() # need to call this for RLlib to make obs and action space accessible
        
        # use dummy env to init single agent observation and action space
        dummy_team_id = list(dummy_env.team_players_map.keys())[0]
        self.observation_space = dummy_env.observation_space(dummy_team_id)
        self.action_space = dummy_env.action_space(dummy_team_id)
        print(f'observation_space: {self.observation_space}')
        print(f'action_space: {self.action_space}')

        # init teams
        self.teams = []
        for team_id, params in teams_config.items():
            team_cls, team_kwargs = params['cls'], params['kwargs']
            team_kwargs['env_config'] = env_config
            team_kwargs['team_id'] = team_id
            for player_idx in range(len(team_kwargs['agents_cls'])):
                team_kwargs['agents_kwargs'][player_idx] = {'observation_space': self.observation_space, 'action_space': self.action_space}
            # team_kwargs['observation_space'] = self.observation_space
            # team_kwargs['action_space'] = self.action_space
            if 'config' not in team_kwargs:
                team_kwargs['config'] = {}
            self.teams.append(team_cls(**team_kwargs))
        print(f'Teams: {self.teams}')

        # update env config with instantiated teams
        for i, team in enumerate(self.teams):
            class Agent(nmmo.Agent):
                name = f'{team.id}'
                policy = f'{team.id}'
            env_config.PLAYERS[i] = Agent
            print(f'env_config.PLAYERS[{i}]: {Agent}')
        print(f'env_config: {env_config}')
                
        # init TeamBasedEnv with env config
        self._env = get_class_from_path(path_to_env_cls)(env_config)
        _ = self._env.reset() # need to call this for RLlib to make obs and action space accessible
        print(f'_env: {self._env}')
        print(f'players: {self._env.players}')
        print(f'player_team_map: {self._env.player_team_map}')
        print(f'team_players_map: {self._env.team_players_map}')
        
        # init agent info
        self._agent_ids = list(self._env.players.keys())
        self.agents = list(self._env.players.values())
        print(f'agents: {self.agents}')
        print(f'_agent_ids: {self._agent_ids}')
        
        self.dones = set()
        
    @override(MultiAgentEnv)
    def observation_space_sample(self, agent_ids: list = None) -> MultiAgentDict:
        if agent_ids is None:
            # agent_ids = list(range(len(self.agents)))
            agent_ids = self._agent_ids
        obs = {agent_id: self.observation_space.sample() for agent_id in agent_ids}

        return obs

    @override(MultiAgentEnv)
    def action_space_sample(self, agent_ids: list = None) -> MultiAgentDict:
        if agent_ids is None:
            # agent_ids = list(range(len(self.agents)))
            agent_ids = self._agent_ids
        actions = {agent_id: self.action_space.sample() for agent_id in agent_ids}

        return actions

    @override(MultiAgentEnv)
    def action_space_contains(self, x: MultiAgentDict) -> bool:
        if not isinstance(x, dict):
            return False
        return all(self.action_space.contains(val) for val in x.values())

    @override(MultiAgentEnv)
    def observation_space_contains(self, x: MultiAgentDict) -> bool:
        if not isinstance(x, dict):
            return False
        return all(self.observation_space.contains(val) for val in x.values())

    @override(MultiAgentEnv)
    def reset(self):
        self.dones = set()
        # return {i: a.reset() for i, a in enumerate(self.agents)}
        
        self.teams = reset_teams(self.teams)
        
        self.agent_id_to_agent = {}
        for team_idx, team in enumerate(self.teams):
            for player_idx, player_id in enumerate(self._env.team_players_map[team_idx]):
                self.agent_id_to_agent[player_id] = team.agents[player_idx]
                
        obs = reset_env(self._env)
        
        # self.agent_id_to_agent = self._env.players
        
        return self._flatten_teams_dict(obs)

    @override(MultiAgentEnv)
    def step(self, action_dict):
        print(f'player to actions:\n{action_dict.keys()}\n{action_dict}')
        
        # convert player dict into team dict
        team_actions = self._unflatten_players_dict(action_dict)
        print(f'team to actions:\n{team_actions.keys()}\n{team_actions}')
        
        # # do post processing of actions
        for team_idx, actions in team_actions.items():
            team_actions[team_idx] = self.post_process_team_actions(self.teams[team_idx], actions)
        
        # step TeamBasedEnv
        obs, rew, done, info = self._env.step(team_actions)
        done["__all__"] = len(self.dones) == len(self.agents)
        print(f'TeamBasedEnv obs keys: {obs.keys()}')
        for key in obs.keys():
            print(f'TeamBasedEnv obs[{key}] keys: {obs[key].keys()}')
        
        # convert team dicts into player dicts
        obs, rew, done, info = [self._flatten_teams_dict(_dict) for _dict in [obs, rew, done, info]]
        print(f'RLlibMultiAgentTeamBasedEnv obs keys: {obs.keys()}')
        
        return obs, rew, done, info
    
    def _flatten_teams_dict(self, _dict):
        flat_dict = {}
        # print(f'dict to flatten: {_dict}')
        for team_idx, player_ids in self._env.team_players_map.items():
            if team_idx in _dict:
                if 'stat' in _dict[team_idx]:
                    stat = _dict[team_idx].pop('stat')
                for player_idx in _dict[team_idx].keys():
                    player_id = player_ids[player_idx]
                    flat_dict[player_id] = _dict[team_idx][player_idx]
        return flat_dict
    
    def _unflatten_players_dict(self, _dict):
        unflat_dict = defaultdict(dict)
        for player_id in _dict.keys():
            team_idx = self._env.player_team_map[player_id]
            player_idx = self._env.team_players_map[team_idx].index(player_id)
            unflat_dict[team_idx][player_idx] = _dict[player_id]
        return unflat_dict
    
    def post_process_team_actions(self, team, actions, verbose=True):
        if verbose:
            print(f'\nteam: {team}')
        for i in actions:
            if verbose:
                print(f'\ti: {i}')
            for atn, args in actions[i].items():
                if verbose:
                    print(f'atn: {atn}')
                    print(f'args: {args}')
                for arg, val in args.items():
                    if verbose:
                        print(f'arg: {arg}')
                        print(f'val: {val}')
                        print(f'edges: {arg.edges}')
                    try:
                        if arg.argType == nmmo.action.Fixed:
                            actions[i][atn][arg] = arg.edges.index(val)
                        elif arg == nmmo.action.Target:
                            actions[i][atn][arg] = self.get_target_index(
                                val, team.agents[i].ob.agents)
                        elif atn in (nmmo.action.Sell,
                                     nmmo.action.Use) and arg == nmmo.action.Item:
                            actions[i][atn][arg] = self.get_item_index(
                                val, team.agents[i].ob.items)
                        elif atn == nmmo.action.Buy and arg == nmmo.action.Item:
                            actions[i][atn][arg] = self.get_item_index(
                                val, team.agents[i].ob.market)
                    except ValueError:
                        # TODO TEMP HACK: RLlib Epoch loop seems to return actions as ints rather than objects, somewhere under hood is this conversion done already?!
                        # need to figure this out
                        actions[i][atn][arg] = val
        return actions

    @staticmethod
    def get_item_index(instance: int, items: np.ndarray) -> int:
        for i, itm in enumerate(items):
            id_ = nmmo.scripting.Observation.attribute(itm,
                                                       nmmo.Serialized.Item.ID)
            if id_ == instance:
                return i
        raise ValueError(f"Instance {instance} not found")

    @staticmethod
    def get_target_index(target: int, agents: np.ndarray) -> int:
        targets = [
            x for x in [
                nmmo.scripting.Observation.attribute(
                    agent, nmmo.Serialized.Entity.ID) for agent in agents
            ] if x
        ]
        return targets.index(target)
    
    
    
    
# env config
path_to_env_cls = 'neurips2022nmmo.TeamBasedEnv'
path_to_env_config_cls = 'deep_nmmo.envs.team_based_env.env_configs.custom_competition_config.CustomCompetitionConfig'
env_config_kwargs = None
# teams_config = {
#     'Combat':
#         {
#             'path_to_team_cls': 'neurips2022nmmo.scripted.CombatTeam'
#         },
#     'Mixture':
#         {
#             'path_to_team_cls': 'neurips2022nmmo.scripted.MixtureTeam'
#         },
# }
# rllib_paths_to_scripted_agents_cls = {
#                                     '0': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '1': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '2': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '3': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '4': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '5': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '6': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     }

num_players_per_team = 8
# will overwrite agents kwargs with observation and action space and player index when initialising teams in RLlib env
custom_team_kwargs = {'agents_cls': [RLlibRandom for _ in range(num_players_per_team)], 'agents_kwargs': [None for _ in range(num_players_per_team)]}

teams_config = {
    # 'RLlib': {'cls': RLlibScriptedHybridAgentTeam, 'kwargs': {'paths_to_scripted_agents_cls': rllib_paths_to_scripted_agents_cls}},
    
    # 'Combat-1': {'cls': neurips2022nmmo.scripted.CombatTeam, 'kwargs': {'team_id': 1}},
    # 'Combat-2': {'cls': neurips2022nmmo.scripted.CombatTeam, 'kwargs': {'team_id': 2}},
    # 'Mixture-1': {'cls': neurips2022nmmo.scripted.MixtureTeam, 'kwargs': {'team_id': 3}},
    
    'R-1': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    'R-2': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    'R-3': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    
    
}
# teams_copies = [1, 2]        
        

env_config = get_class_from_path(path_to_env_config_cls)()
ma_env = RLlibMultiAgentTeamBasedEnv(env_config=env_config,
                                     # path_to_env_cls=path_to_env_cls,
                                     # path_to_env_config_cls=path_to_env_config_cls,
                                     teams_config=teams_config)
observations = ma_env.reset()
teams, agent_id_to_agent = ma_env.teams, ma_env.agent_id_to_agent
print(f'')
# print(f'RESET\nobs:\n{obs}\nteams:\n{teams}')
print(f'Reset teams and obs')
step_counter = 1
while observations:
    print(f'\nStep {step_counter}')
    print(f'Step observation keys: {observations.keys()}')
    # agent_to_actions = {agent_id: agent(observations[agent_id]) for agent_id, agent in agent_id_to_agent.items()}
    
    agent_to_actions = {}
    for agent_id in observations.keys():
        # print(f'\nagent_id: {agent_id}')
        # print(f'agent: {agent}')
        agent = agent_id_to_agent[agent_id]
        # actions = agent(observations[agent_id])
        # actions = agent.compute_single_action(observations[agent_id])
        actions, state_outs, info = agent.compute_actions(observations[agent_id])
        print(f'agent ob: {agent.ob}')
        # print(f'actions: {actions}')
        agent_to_actions[agent_id] = actions
    
    observations, rewards, dones, infos = ma_env.step(agent_to_actions)
    raise Exception(f'Completed step.')
    step_counter += 1
    
print(f'\nEPISODE COMPLETED!!!')

  5%|████████▎                                                                                                                                                              | 2/40 [00:00<00:02, 14.62it/s]

Generating 40 maps


  5%|████████▎                                                                                                                                                              | 2/40 [00:00<00:02, 15.62it/s]

observation_space: Dict(Entity:Dict(Continuous:Box(-1048576.0, 1048576.0, (100, 24), float32), Discrete:Box(0, 4096, (100, 5), int32), N:Box(0, 100, (1,), int32)), Item:Dict(Continuous:Box(-1048576.0, 1048576.0, (170, 16), float32), Discrete:Box(0, 4096, (170, 3), int32), N:Box(0, 170, (1,), int32)), Market:Dict(Continuous:Box(-1048576.0, 1048576.0, (170, 16), float32), Discrete:Box(0, 4096, (170, 3), int32), N:Box(0, 170, (1,), int32)), Tile:Dict(Continuous:Box(-1048576.0, 1048576.0, (225, 4), float32), Discrete:Box(0, 4096, (225, 3), int32), N:Box(0, 15, (1,), int32)))
action_space: Dict(<class 'nmmo.io.action.Attack'>:Dict(<class 'nmmo.io.action.Style'>:Discrete(3), <class 'nmmo.io.action.Target'>:Discrete(100)), <class 'nmmo.io.action.Buy'>:Dict(<class 'nmmo.io.action.Item'>:Discrete(170)), <class 'nmmo.io.action.Comm'>:Dict(<class 'nmmo.io.action.Token'>:Discrete(170)), <class 'nmmo.io.action.Move'>:Dict(<class 'nmmo.io.action.Direction'>:Discrete(4)), <class 'nmmo.io.action.Sell'

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 15.78it/s]


_env: <neurips2022nmmo.env.team_based_env.TeamBasedEnv object at 0x7e965c2ba5e0>
players: {1: <nmmo.entity.player.Player object at 0x7e96549861f0>, 2: <nmmo.entity.player.Player object at 0x7e9654d1d130>, 3: <nmmo.entity.player.Player object at 0x7e9654ebd070>, 4: <nmmo.entity.player.Player object at 0x7e9654ebdf70>, 5: <nmmo.entity.player.Player object at 0x7e9654ea5eb0>, 6: <nmmo.entity.player.Player object at 0x7e965517fdf0>, 7: <nmmo.entity.player.Player object at 0x7e965516cd30>, 8: <nmmo.entity.player.Player object at 0x7e96554f4c70>, 9: <nmmo.entity.player.Player object at 0x7e96554c2bb0>, 10: <nmmo.entity.player.Player object at 0x7e9655df6af0>, 11: <nmmo.entity.player.Player object at 0x7e9655fb9a30>, 12: <nmmo.entity.player.Player object at 0x7e9655fa2970>, 13: <nmmo.entity.player.Player object at 0x7e96561d48b0>, 14: <nmmo.entity.player.Player object at 0x7e96563f77f0>, 15: <nmmo.entity.player.Player object at 0x7e96563e8730>, 16: <nmmo.entity.player.Player object at 0x7e965

KeyError: 0

## TODO

- Get `RLlibMultiAgentTeamBasedEnv` working with `ray` and `rllib`
    - Start by trying to use non-learning policies i.e. just map to scripted agents
        - PROBLEM: env seems to be empty of players and teams after initialisation when run with RLlib. Why?
- Use `RLlibScriptedHybridAgentTeam` to try and put a learning agent into the environment and train it

In [338]:
path_to_rllib_trainer_cls = 'ray.rllib.agents.ppo.PPOTrainer'

env_config = get_class_from_path(path_to_env_config_cls)()

policies = {}
policy_id = 1
num_teams, num_players_per_team = 3, 8
for _ in range(num_teams):
    player_idx = 0
    for _ in range(num_players_per_team):
        policies[str(policy_id)] = PolicySpec(
                                    policy_class=RLlibRandom, # infer from env
                                    # observation_space=dummy_env.observation_space(player_id),
                                    # action_space=dummy_env.action_space(player_id),
                                    config={'env_config': env_config, 'idx': player_idx},
                                )
        policy_id += 1
        player_idx += 1

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    '''Maps agent ID (int) to corresponding policy ID (str) of policy which agent should use.'''
    print(f'>>> Mapping agent_id {agent_id} to {str(agent_id)}')
    return str(agent_id)

multiagent_config = {
    'policies': policies,
    'policy_mapping_fn': policy_mapping_fn,
    'policies_to_train': [],
}

env_config = {
    # 'path_to_env_cls': path_to_env_cls,
    # 'path_to_env_config_cls': path_to_env_config_cls,
    'env_config': env_config,
    'teams_config': teams_config,
}


rllib_config = {
    'env': 'ma_env',
    
    'env_config': env_config,
    
    'disable_env_checking': True,
    # 'disable_env_checking': False,
    
    'framework': 'torch',
    
    'multiagent': multiagent_config,
    
}

In [339]:
# register env with ray
register_env('ma_env', lambda env_config: RLlibMultiAgentTeamBasedEnv(**env_config))

In [340]:
# merge rllibg trainer's default config with specified config
path_to_agent = '.'.join(path_to_rllib_trainer_cls.split('.')[:-1])
_rllib_config = get_module_from_path(path_to_agent).DEFAULT_CONFIG.copy()
_rllib_config.update(rllib_config)
print(_rllib_config)

# init rllib trainer
trainer = get_class_from_path(path_to_rllib_trainer_cls)(config=_rllib_config)
print(f'Initialised trainer')

2022-09-04 22:03:13,032	WARNING multi_agent.py:121 -- `config.multiagent.policies_to_train` is empty! Make sure - if you would like to learn at least one policy - to add its ID to that list.


{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': 'ma_env', 'env_config': {'env_config': <deep_nmmo.envs.team_based_env.env_configs.custom_competition_config.CustomCompetitionConfig object at 0x7e965db2e160>, 'teams_config': {'R-1': {'cls': <class 'deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam'>, 'kwargs': {'agents_cls': [<class '__main__.RLlibRandom'>, <class '__main__.RLlibRandom'>, <class '__main__.RLlibRandom'>, <class '__

  0%|          | 0/40 [00:00<?, ?it/s]m 


(RolloutWorker pid=477524) Generating 40 maps
(RolloutWorker pid=477525) Generating 40 maps


100%|██████████| 40/40 [00:02<00:00, 14.73it/s]


...

(RolloutWorker pid=477525) observation_space: Dict(Entity:Dict(Continuous:Box(-1048576.0, 1048576.0, (100, 24), float32), Discrete:Box(0, 4096, (100, 5), int32), N:Box(0, 100, (1,), int32)), Item:Dict(Continuous:Box(-1048576.0, 1048576.0, (170, 16), float32), Discrete:Box(0, 4096, (170, 3), int32), N:Box(0, 170, (1,), int32)), Market:Dict(Continuous:Box(-1048576.0, 1048576.0, (170, 16), float32), Discrete:Box(0, 4096, (170, 3), int32), N:Box(0, 170, (1,), int32)), Tile:Dict(Continuous:Box(-1048576.0, 1048576.0, (225, 4), float32), Discrete:Box(0, 4096, (225, 3), int32), N:Box(0, 15, (1,), int32)))
(RolloutWorker pid=477525) action_space: Dict(<class 'nmmo.io.action.Attack'>:Dict(<class 'nmmo.io.action.Style'>:Discrete(3), <class 'nmmo.io.action.Target'>:Discrete(100)), <class 'nmmo.io.action.Buy'>:Dict(<class 'nmmo.io.action.Item'>:Discrete(170)), <class 'nmmo.io.action.Comm'>:Dict(<class 'nmmo.io.action.Token'>:Discrete(170)), <class 'nmmo.io.action.Move'>:Dict(<class 'nmmo.io.action.

100%|██████████| 40/40 [00:02<00:00, 15.39it/s]


(RolloutWorker pid=477524) _env: <neurips2022nmmo.env.team_based_env.TeamBasedEnv object at 0x7ef2c64172e0>
(RolloutWorker pid=477524) players: {1: <nmmo.entity.player.Player object at 0x7ef2bcfb4550>, 2: <nmmo.entity.player.Player object at 0x7ef2bda0d490>, 3: <nmmo.entity.player.Player object at 0x7ef2bcfdc3d0>, 4: <nmmo.entity.player.Player object at 0x7ef2bdeb8310>, 5: <nmmo.entity.player.Player object at 0x7ef2bdea5250>, 6: <nmmo.entity.player.Player object at 0x7ef2be175190>, 7: <nmmo.entity.player.Player object at 0x7ef2be1660d0>, 8: <nmmo.entity.player.Player object at 0x7ef2be166fd0>, 9: <nmmo.entity.player.Player object at 0x7ef2be1f5f10>, 10: <nmmo.entity.player.Player object at 0x7ef2be37ee50>, 11: <nmmo.entity.player.Player object at 0x7ef2be36bd90>, 12: <nmmo.entity.player.Player object at 0x7ef2be5bfcd0>, 13: <nmmo.entity.player.Player object at 0x7ef2be5abc10>, 14: <nmmo.entity.player.Player object at 0x7ef2be636b50>, 15: <nmmo.entity.player.Player object at 0x7ef2be600

2022-09-04 22:03:24,814	INFO trainable.py:160 -- Trainable.setup took 11.785 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-09-04 22:03:24,815	WARNING util.py:65 -- Install gputil for GPU system monitoring.


Initialised trainer


In [341]:
# perform one training epoch
results = trainer.train()

2022-09-04 22:03:25,046	WARNING algorithm.py:2178 -- Worker crashed during training or evaluation! To try to continue without failed worker(s), set `ignore_worker_failures=True`. To try to recover the failed worker(s), set `recreate_failed_workers=True`.


(RolloutWorker pid=477524) >>> Mapping agent_id 1 to 1


RayTaskError(TypeError): [36mray::RolloutWorker.sample()[39m (pid=477525, ip=128.40.41.23, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7fca56f06b80>)
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 806, in sample
    batches = [self.input_reader.next()]
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", line 92, in next
    batches = [self.get_data()]
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", line 282, in get_data
    item = next(self._env_runner)
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", line 707, in _env_runner
    eval_results = _do_policy_eval(
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", line 1207, in _do_policy_eval
    eval_results[policy_id] = policy.compute_actio

(RolloutWorker pid=477524) >>> Mapping agent_id 2 to 2
(RolloutWorker pid=477524) >>> Mapping agent_id 3 to 3
(RolloutWorker pid=477524) >>> Mapping agent_id 4 to 4
(RolloutWorker pid=477524) >>> Mapping agent_id 5 to 5
(RolloutWorker pid=477524) >>> Mapping agent_id 6 to 6
(RolloutWorker pid=477524) >>> Mapping agent_id 7 to 7
(RolloutWorker pid=477524) >>> Mapping agent_id 8 to 8
(RolloutWorker pid=477524) >>> Mapping agent_id 9 to 9
(RolloutWorker pid=477524) >>> Mapping agent_id 10 to 10
(RolloutWorker pid=477524) >>> Mapping agent_id 11 to 11
(RolloutWorker pid=477524) >>> Mapping agent_id 12 to 12
(RolloutWorker pid=477524) >>> Mapping agent_id 13 to 13
(RolloutWorker pid=477524) >>> Mapping agent_id 14 to 14
(RolloutWorker pid=477524) >>> Mapping agent_id 15 to 15
(RolloutWorker pid=477524) >>> Mapping agent_id 16 to 16
(RolloutWorker pid=477524) >>> Mapping agent_id 17 to 17
(RolloutWorker pid=477524) >>> Mapping agent_id 18 to 18
(RolloutWorker pid=477524) >>> Mapping agent_id

2022-09-04 22:03:30,283	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RolloutWorker.sample() (pid=477524, ip=128.40.41.23, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7ef2c7404bb0>)
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 806, in sample
    batches = [self.input_reader.next()]
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", line 92, in next
    batches = [self.get_data()]
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", line 282, in get_data
    item = next(self._env_runner)
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", line 707, in _env_runner
    eval_results = _do_policy_eval(
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", lin

In [72]:


class RLlibScriptedHybridAgentTeam(Team):
    def __init__(self,
                 team_id,
                 env_config,
                 paths_to_scripted_agents_cls,
                 **kwargs):
        super().__init__(team_id, env_config)
        self.id = team_id
        print(f'team_id: {self.id}')
        
        self.scripted_agents = [get_class_from_path(path_to_scripted_agent_cls)(config=env_config, idx=int(idx)) for idx, path_to_scripted_agent_cls in paths_to_scripted_agents_cls.items()]
        self.scripted_agent_ids = set([int(idx) for idx in paths_to_scripted_agents_cls.keys()])
        self.scripted_agent_idxs = set([idx for idx in range(len(self.scripted_agents))])
        
        self.rllib_agents = [None]
        self.rllib_agent_ids = set([sorted(self.scripted_agent_ids)[-1]+1])
        self.rllib_agent_idxs = set([len(self.scripted_agents)+idx for idx in range(1, len(self.rllib_agents)+1)])
        
        print(f'scripted_agents: {self.scripted_agents}')
        print(f'scripted_agent_ids: {self.scripted_agent_ids}')
        print(f'scripted_agent_idxs: {self.scripted_agent_idxs}')
        
        print(f'rllib_agents: {self.rllib_agents}')
        print(f'rllib_agent_ids: {self.rllib_agent_ids}')
        print(f'rllib_agent_idxs: {self.rllib_agent_idxs}')
        
        # init multi-agent env for team to interact with
        agent_ids_to_agents = {}
        for agent_id, agent in zip(self.scripted_agent_ids, self.scripted_agents):
            agent_ids_to_agents[agent_id] = agent
        for agent_id, agent in zip(self.rllib_agent_ids, self.rllib_agents):
            agent_ids_to_agents[agent_id] = agent
        self.team_env = RLlibMultiAgentTeamEnv(
                                            env_config=env_config,
                                            agent_ids_to_agents=agent_ids_to_agents,
                                            team_id=self.id,
                                        )
        print(f'team_env: {self.team_env}')
        
    def reset(self, *args, **kwargs):
        pass
    
    def get_rllib_agent_actions(self, observations):
        return {i: self.agents[i](obs) for i, obs in observations.items() if i in self.rllib_agent_idxs}
    
    def get_scripted_agent_actions(self, observations):
        return {i: self.agents[i](obs) for i, obs in observations.items() if i in self.scripted_agent_idxs}
    
    def post_process_actions(self, actions):
        for i in actions:
            for atn, args in actions[i].items():
                for arg, val in args.items():
                    if arg.argType == nmmo.action.Fixed:
                        actions[i][atn][arg] = arg.edges.index(val)
                    elif arg == nmmo.action.Target:
                        actions[i][atn][arg] = self.get_target_index(
                            val, self.agents[i].ob.agents)
                    elif atn in (nmmo.action.Sell,
                                 nmmo.action.Use) and arg == nmmo.action.Item:
                        actions[i][atn][arg] = self.get_item_index(
                            val, self.agents[i].ob.items)
                    elif atn == nmmo.action.Buy and arg == nmmo.action.Item:
                        actions[i][atn][arg] = self.get_item_index(
                            val, self.agents[i].ob.market)
        return actions
    
    def act(self, observations, rewards=None):
        '''
        During training, pass rewards from last step to update RLlib policy.
        
        During inference (e.g. when make submission), no rewards need to be
        passed to act().
        '''
        if "stat" in observations:
            stat = observations.pop("stat")
            
        if rewards is None:
            # agent is training, register rewards
            # if an action was chosen at the last step, assign team reward for taking that action
            # TODO
            pass
        else:
            # not training, no need to consider rewards
            pass
        
        # TODO
        # get team actions for this step
        actions = {}
        actions.update(self.get_scripted_agent_actions(observations))
        actions.update(self.get_rllib_agent_actions(observations))
        
        # return team actions to TeamBasedEnv
        return self.post_process_actions(actions)

    @staticmethod
    def get_item_index(instance: int, items: np.ndarray) -> int:
        for i, itm in enumerate(items):
            id_ = nmmo.scripting.Observation.attribute(itm,
                                                       nmmo.Serialized.Item.ID)
            if id_ == instance:
                return i
        raise ValueError(f"Instance {instance} not found")

    @staticmethod
    def get_target_index(target: int, agents: np.ndarray) -> int:
        targets = [
            x for x in [
                nmmo.scripting.Observation.attribute(
                    agent, nmmo.Serialized.Entity.ID) for agent in agents
            ] if x
        ]
        return targets.index(target)
        
        
        
        
        
# env config
path_to_env_cls = 'neurips2022nmmo.TeamBasedEnv'
path_to_env_config_cls = 'deep_nmmo.envs.team_based_env.env_configs.custom_competition_config.CustomCompetitionConfig'
env_config_kwargs = None
# teams_config = {
#     'Combat':
#         {
#             'path_to_team_cls': 'neurips2022nmmo.scripted.CombatTeam'
#         },
#     'Mixture':
#         {
#             'path_to_team_cls': 'neurips2022nmmo.scripted.MixtureTeam'
#         },
# }
rllib_paths_to_scripted_agents_cls = {
                                    '0': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '1': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '2': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '3': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '4': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '5': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '6': 'neurips2022nmmo.scripted.baselines.Mage',
                                    }

teams_config = {
    'RLlib': {'cls': RLlibScriptedHybridAgentTeam, 'kwargs': {'paths_to_scripted_agents_cls': rllib_paths_to_scripted_agents_cls}},
    'Combat': {'cls': neurips2022nmmo.scripted.CombatTeam, 'kwargs': {}},
    'Mixture': {'cls': neurips2022nmmo.scripted.MixtureTeam, 'kwargs': {}},
}
teams_copies = [1, 2]        
        
        
        
        
        
# init env params
env_config = get_class_from_path(path_to_env_config_cls)()

teams = []
for team_id, params in teams_config.items():
    team_cls, team_kwargs = params['cls'], params['kwargs']
    team_kwargs['env_config'] = env_config
    team_kwargs['team_id'] = team_id
    teams.append(team_cls(**team_kwargs))
print(f'Teams: {teams}')

for i, team in enumerate(teams):
    class Agent(nmmo.Agent):
        name = f'{team.id}'
        policy = f'{team.id}'
    env_config.PLAYERS[i] = Agent

  5%|████████▎                                                                                                                                                              | 2/40 [00:00<00:02, 14.91it/s]

team_id: RLlib | kwargs: {'paths_to_scripted_agents_cls': {'0': 'neurips2022nmmo.scripted.baselines.Mage', '1': 'neurips2022nmmo.scripted.baselines.Mage', '2': 'neurips2022nmmo.scripted.baselines.Mage', '3': 'neurips2022nmmo.scripted.baselines.Mage', '4': 'neurips2022nmmo.scripted.baselines.Mage', '5': 'neurips2022nmmo.scripted.baselines.Mage', '6': 'neurips2022nmmo.scripted.baselines.Mage'}}
team_id: RLlib
scripted_agents: [<neurips2022nmmo.scripted.baselines.Mage object at 0x7f252bfcc610>, <neurips2022nmmo.scripted.baselines.Mage object at 0x7f252bfcc2e0>, <neurips2022nmmo.scripted.baselines.Mage object at 0x7f252bfcc2b0>, <neurips2022nmmo.scripted.baselines.Mage object at 0x7f252bfcc3a0>, <neurips2022nmmo.scripted.baselines.Mage object at 0x7f252bfcc310>, <neurips2022nmmo.scripted.baselines.Mage object at 0x7f252bfcc400>, <neurips2022nmmo.scripted.baselines.Mage object at 0x7f252bfcc190>]
scripted_agent_ids: {0, 1, 2, 3, 4, 5, 6}
scripted_agent_idxs: {0, 1, 2, 3, 4, 5, 6}
rllib_agen

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 14.97it/s]


observation_space: Dict(Entity:Dict(Continuous:Box(-1048576.0, 1048576.0, (100, 24), float32), Discrete:Box(0, 4096, (100, 5), int32), N:Box(0, 100, (1,), int32)), Item:Dict(Continuous:Box(-1048576.0, 1048576.0, (170, 16), float32), Discrete:Box(0, 4096, (170, 3), int32), N:Box(0, 170, (1,), int32)), Market:Dict(Continuous:Box(-1048576.0, 1048576.0, (170, 16), float32), Discrete:Box(0, 4096, (170, 3), int32), N:Box(0, 170, (1,), int32)), Tile:Dict(Continuous:Box(-1048576.0, 1048576.0, (225, 4), float32), Discrete:Box(0, 4096, (225, 3), int32), N:Box(0, 15, (1,), int32)))
action_space: Dict(<class 'nmmo.io.action.Attack'>:Dict(<class 'nmmo.io.action.Style'>:Discrete(3), <class 'nmmo.io.action.Target'>:Discrete(100)), <class 'nmmo.io.action.Buy'>:Dict(<class 'nmmo.io.action.Item'>:Discrete(170)), <class 'nmmo.io.action.Comm'>:Dict(<class 'nmmo.io.action.Token'>:Discrete(170)), <class 'nmmo.io.action.Move'>:Dict(<class 'nmmo.io.action.Direction'>:Discrete(4)), <class 'nmmo.io.action.Sell'

In [48]:
# set up configs

# env config
path_to_env_cls = 'neurips2022nmmo.TeamBasedEnv'
path_to_env_config_cls = 'deep_nmmo.envs.team_based_env.env_configs.custom_competition_config.CustomCompetitionConfig'
env_config_kwargs = None
teams_config = {
    'Combat':
        {
            'path_to_team_cls': 'neurips2022nmmo.scripted.CombatTeam'
        },
    'Mixture':
        {
            'path_to_team_cls': 'neurips2022nmmo.scripted.MixtureTeam'
        },
}
teams_copies = [1, 2]

# init env params
env_config, teams_copies, teams = init_env_params(path_to_env_config_cls=path_to_env_config_cls,
                                                  env_config_kwargs=env_config_kwargs,
                                                  teams_copies=teams_copies,
                                                  teams_config=teams_config)



# rllib config
path_to_rllib_trainer_cls = 'ray.rllib.agents.ppo.PPOTrainer'

# ma_cls = make_multi_agent(lambda env_config: get_class_from_path(path_to_env_cls)(env_config))
# print(ma_cls)
# dummy_env = ma_cls(env_config)
# print(dummy_env)
# print(type(dummy_env))
# _ = dummy_env.reset()


dummy_env = RLlibMultiAgentTeamBasedEnv(
                                    path_to_env_cls=path_to_env_cls,
                                    path_to_env_config_cls=path_to_env_config_cls,
                                    teams_copies=teams_copies,
                                    teams_config=teams_config,
                                )
print(dummy_env)

policies = {}
for policy_id in dummy_env._agent_ids:
    policies[str(policy_id)] = PolicySpec(
                                # policy_class=player.__class__, # infer automatically from algorithm
                                # observation_space=dummy_env.observation_space(player_id),
                                # action_space=dummy_env.action_space(player_id),
                                config={},
                            )

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    '''Maps agent ID to corresponding policy ID of policy which agent should use.'''
    return str(agent_id)

multiagent_config = {
    'policies': policies,
    'policy_mapping_fn': policy_mapping_fn,
}


rllib_config = {
    'framework': 'torch',
    
    'multiagent': multiagent_config,
    
}

  5%|████████▎                                                                                                                                                              | 2/40 [00:00<00:02, 14.90it/s]

Generating 40 maps


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 15.15it/s]


agents: [<nmmo.entity.player.Player object at 0x7f543a832a60>, <nmmo.entity.player.Player object at 0x7f544fe559a0>, <nmmo.entity.player.Player object at 0x7f54502fe8e0>, <nmmo.entity.player.Player object at 0x7f5439637820>, <nmmo.entity.player.Player object at 0x7f5439624760>, <nmmo.entity.player.Player object at 0x7f54398b46a0>, <nmmo.entity.player.Player object at 0x7f54398a55e0>, <nmmo.entity.player.Player object at 0x7f5439a6e520>, <nmmo.entity.player.Player object at 0x7f5439a4c460>, <nmmo.entity.player.Player object at 0x7f543a0ef3a0>, <nmmo.entity.player.Player object at 0x7f543a1b62e0>, <nmmo.entity.player.Player object at 0x7f543a1a6220>, <nmmo.entity.player.Player object at 0x7f543a5b5160>, <nmmo.entity.player.Player object at 0x7f543a5a70a0>, <nmmo.entity.player.Player object at 0x7f543a5a7fa0>, <nmmo.entity.player.Player object at 0x7f544ff07ee0>, <nmmo.entity.player.Player object at 0x7f544feebe20>, <nmmo.entity.player.Player object at 0x7f544fffcd60>, <nmmo.entity.player

TypeError: 'NoneType' object is not callable

In [ ]:
# register env with ray
register_env(path_to_env_cls.split('.')[-1], lambda env_config: make_multi_agent(get_class_from_path(path_to_env_cls))(env_config))

In [14]:
# # update rllib config with observation and action spaces
# dummy_env = get_class_from_path(path_to_env_cls)(env_config)
# agent = 0
# rllib_config['observation_space'] = dummy_env.observation_space(agent)
# rllib_config['action_space'] = dummy_env.action_space(agent)

# init rllib trainer
trainer = get_class_from_path(path_to_rllib_trainer_cls)(config=rllib_config)

RayTaskError(AssertionError): [36mray::RolloutWorker.apply()[39m (pid=67793, ip=128.40.41.23, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f0f6846fb50>)
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1664, in apply
    return func(self, *args, **kwargs)
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 269, in <lambda>
    self.foreach_worker(lambda w: w.assert_healthy())
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 766, in assert_healthy
    assert is_healthy, (
AssertionError: RolloutWorker <ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f0f6846fb50> (idx=1; num_workers=2) not healthy!

In [ ]:
env = TeamBasedEnv(env_config)

In [ ]:
print(env.observation_space(0))
print(type(env.observation_space(0)))

2022-09-01 13:22:29,162	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RolloutWorker.apply() (pid=67794, ip=128.40.41.23, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f86f6178a90>)
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1664, in apply
    return func(self, *args, **kwargs)
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 269, in <lambda>
    self.foreach_worker(lambda w: w.assert_healthy())
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 766, in assert_healthy
    assert is_healthy, (
AssertionError: RolloutWorker <ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f86f6178a90> (idx=2; num_workers=2) not healthy!


In [ ]:
for key, val in env.observation_space(0).items():
    print(f'\nkey: {key}')
    print(f'val: {val}')

2022-09-01 13:08:06,043	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RolloutWorker.apply() (pid=65655, ip=128.40.41.23, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f07c37f8b50>)
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1664, in apply
    return func(self, *args, **kwargs)
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 269, in <lambda>
    self.foreach_worker(lambda w: w.assert_healthy())
  File "/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 766, in assert_healthy
    assert is_healthy, (
AssertionError: RolloutWorker <ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f07c37f8b50> (idx=2; num_workers=2) not healthy!
